In [314]:
import pandas as pd
import math

In [315]:
data = pd.read_csv('cars.csv')
data.head()

,buying,maint,doors,persons,lug_boot,safety,clazz
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [316]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [317]:
classes = list(data['clazz'].unique())
features = list(data.keys())[:6]

In [318]:
def create_table(data):
    table = {}

    value_counts = data['clazz'].value_counts()
    table['class_names'] = list(value_counts.index)
    table['class_counts'] = list(value_counts.values)

    for feature in features:
        table[feature] = {}

        counts = data.groupby('clazz')[feature].value_counts()
        data_counts = counts.unstack('clazz')

        # correction factor

        # if data_counts.isna().any(axis=None):
        #     data_counts.fillna(value=0, inplace=True)
        #     data_counts += 1
        
        data_probabilities = data_counts / data_counts.sum()

        for value in data_probabilities.index:
            probabilities = data_probabilities.loc[value]
            probs = {}
            for i in classes:
                probs[i] = 0
            for i in classes:
                if not math.isnan(probabilities[i]):
                    probs[i] = probabilities[i]
            table[feature][value] = probs
            
    return table  

In [319]:
table = create_table(train)

In [320]:
def make_prediction(row, table):
    class_estimates = dict()
    for i in range(len(classes)):
        class_estimates[table['class_names'][i]] = table['class_counts'][i]
    for feature in features:
        value = row[feature]
        probabilities = table[feature][value]
        for i in classes:
            class_estimates[i] *= probabilities[i]
    max_value = 0
    prediction = ""
    for i in class_estimates.keys():
        if max_value < class_estimates[i]:
            max_value = class_estimates[i]
            prediction = i
    return prediction

In [321]:
correct = 0
for i in range(len(test)):
    current_row = test.iloc[i]
    prediction = make_prediction(current_row, table)
    if current_row['clazz'] == prediction:
        correct += 1
print("Accuracy is ", correct / len(test))

Accuracy is  0.8641618497109826
